In [1]:
%cd ../
%matplotlib inline

/home/zulqarnain/featuresounddepression/htfatorch


In [2]:
import logging
import numpy as np
import htfa_torch.htfa as HTFA
import htfa_torch.niidb as niidb
import htfa_torch.utils as utils

/home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

In [4]:
lepping_db = niidb.FMriActivationsDb('data/sound_depression.db', mask='/data/zulqarnain/sound_depression/fmriprep/all_masks/wholebrain.nii.gz')

In [5]:
htfa = HTFA.HierarchicalTopographicFactorAnalysis(list(lepping_db.all(_limit=2)), mask='/data/zulqarnain/sound_depression/fmriprep/all_masks/wholebrain.nii.gz', num_factors=100)

01/12/2019 22:21:58 Loading Nifti image /data/zulqarnain/sound_depression/fmriprep/sub-mdd16/func/sub-mdd16_task-music_run-5_bold_space-MNI152NLin2009cAsym_preproc.nii.gz with mask /data/zulqarnain/sound_depression/fmriprep/all_masks/wholebrain.nii.gz (zscore=True, smooth=6)


In [6]:
htfa.normalize_activations()

[tensor(4.8044), tensor(4.8044)]

In [7]:
losses = htfa.train(num_steps=2, learning_rate=1e-2, log_level=logging.INFO, num_particles=1,
                    batch_size=300, use_cuda=True, checkpoint_steps=50, blocks_batch_size=4)

RuntimeError: CUDA error: out of memory (malloc at /opt/conda/conda-bld/pytorch_1544174967633/work/aten/src/THC/THCCachingAllocator.cpp:205)
frame #0: c10::Error::Error(c10::SourceLocation, std::string const&) + 0x45 (0x7f242a823cc5 in /home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.6/site-packages/torch/lib/libc10.so)
frame #1: <unknown function> + 0x133717f (0x7f242e2f117f in /home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.6/site-packages/torch/lib/libcaffe2_gpu.so)
frame #2: <unknown function> + 0x133798a (0x7f242e2f198a in /home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.6/site-packages/torch/lib/libcaffe2_gpu.so)
frame #3: at::native::empty_cuda(c10::ArrayRef<long>, at::TensorOptions const&) + 0x2d6 (0x7f242f6db9c6 in /home/zulqarnain/anaconda3/envs/htfatorch/lib/python3.6/site-packages/torch/lib/libcaffe2_gpu.so)
frame #4: at::CUDAFloatType::empty(c10::ArrayRef<long>, at::TensorOptions const&) const + 0x161 (0x7f242e202b21 in /home/zulqarnain/anaconda3/envs/htfat

In [ ]:
utils.plot_losses(losses)

In [ ]:
for b in np.random.choice(range(dtfa.num_blocks), 5):
    dtfa.plot_original_brain(b)
    dtfa.plot_reconstruction(b)

In [ ]:
def task_labeler(b):
    return b.task

In [ ]:
dtfa.scatter_task_embedding(labeler=task_labeler, figsize=None)

In [ ]:
def subject_labeler(block):
    return 'Subject %d' % block.subject

def subject_run_labeler(block):
    return 'sub%drun%d' % (block.subject, block.run)

In [ ]:
dtfa.scatter_subject_embedding(labeler=subject_labeler, figsize=None)

In [ ]:
dtfa.visualize_factor_embedding()

In [ ]:
dtfa.average_reconstruction_error()